In [34]:
import pandas as pd
from catboost import CatBoostClassifier, cv, Pool
from datetime import date, timedelta
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [35]:
df = pd.read_csv("dataset.csv")

df["partnerrolestart_date"] = df["partnerrolestart_date"].astype("datetime64[ns]")
df["client_start_date"] = df["client_start_date"].astype("datetime64[ns]")

cutoff_train_date = pd.to_datetime(date(2020, 9, 1))
cutoff_test_date = pd.to_datetime(date(2020, 12, 1))
min_date = df["client_start_date"].min()
num_weeks = (cutoff_train_date - min_date).days // 7 + 1

Дальше идёт создание как раз-таки датасета и генерация признаков

In [37]:
def weeks_count(lst_dates: list) -> pd.Series:
    lst_weeks = [0 for _ in range(num_weeks)]
    for item in lst_dates:
        lst_weeks[(item - min_date).days // 7] += 1
    return pd.Series(lst_weeks)


def max_length(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter_max = 0
    counter = 0
    for element in row:
        if element > 0:
            counter += 1
        counter_max = max(counter, counter_max)
        if element == 0:
            counter = 0
    return counter_max


def max_length_increasing(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter_max = 0
    counter = 0
    prev_sequence = [1]
    for element in row:
        if element >= max(prev_sequence):
            counter += 1
        counter_max = max(counter, counter_max)
        if element < max(prev_sequence):
            counter = 0
            prev_sequence = [1]
        prev_sequence.append(element)
    return counter_max


def max_length_zeros(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter_max = 0
    counter = 0
    for element in row:
        if element == 0:
            counter += 1
        counter_max = max(counter, counter_max)
        if element != 0:
            counter = 0
    return counter_max

def num_nunzero_streaks(row: pd.Series) -> int:
    row = row.loc['week_1':'week_79']
    counter = 0
    lst_nnz = []
    for element in row:
        if element != 0:
            lst_nnz.append(element)
        else:
            counter += (len(lst_nnz) >= 5)
            lst_nnz = []
    return counter


def recent_clients(row: pd.Series) -> int:
    row = row.loc['week_70':'week_79']
    return sum(row)

In [38]:
df_train_cutoff = df[df["client_start_date"] < cutoff_train_date]
data_train = (
    df_train_cutoff.groupby("clientbankpartner_pin")
    .agg(
        {
            "client_pin": "count",
            "partner_src_type_ccode": "first",
            "partnerrolestart_date": "first",
            "client_start_date": lambda x: list(x),
        }
    )
    .reset_index()
)

data_train.rename(
    {
        "clientbankpartner_pin": "partner_pin",
        "client_pin": "num_clients",
        "partner_src_type_ccode": "partner_type",
        "partnerrolestart_date": "partner_start_date",
        "client_start_date": "lst_dates_involving",
    },
    axis=1,
    inplace=True,
)

time_series_train = data_train["lst_dates_involving"].apply(weeks_count)
time_series_train.columns = [f"week_{i+1}" for i in range(79)]
data_train = pd.concat([data_train, time_series_train], axis=1)

data_train = data_train.assign(
    time_as_partner=(cutoff_train_date - data_train["partner_start_date"]).dt.days,
    since_last_client=data_train["lst_dates_involving"].apply(lambda x: (cutoff_train_date - max(x)).days),
    max_length=data_train.apply(max_length, axis=1),
    max_length_increasing=data_train.apply(max_length_increasing, axis=1),
    max_length_zeros=data_train.apply(max_length_zeros, axis=1),
    recent_clients=data_train.apply(recent_clients, axis=1)
)

Тут идёт трейн)

In [39]:
def create_time_features(df,report_date = '2020-9-01'):
    df['month'] = df['client_start_date'].dt.month
    df['day'] = df['client_start_date'].dt.day
    df['week'] = df['client_start_date'].dt.day.map(lambda x: x // 7)
    df['minute'] = df['client_start_date'].dt.minute
    df['year'] = df['client_start_date'].dt.year

    df['dayofweek'] = df['client_start_date'].dt.dayofweek
    df['all_year_time'] = df['client_start_date'].apply(lambda x: (x.year - 2019) * 365 + x.month * 30 + x.day)
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')
    
    df['time_to_report'] = pd.to_datetime(report_date) - df['client_start_date']
    df['days_to_report'] = df['time_to_report'].dt.days.astype(int)
    
    return df

df_train_cutoff = create_time_features(df_train_cutoff)

def distance_calc(x):
    dist = []
    for i in range(len(x)-1):
        dist += [x[i+1] - x[i]]
    return dist

def get_time_features(df):
    time_day_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(['mean', 'std', 'min', 'max', 'median','count'])
    time_day_features.columns = [f'time_day_features_{c}' for c in time_day_features.columns]
    time_day_features['time_day_features_diff'] = time_day_features['time_day_features_max'] - time_day_features['time_day_features_min']
    time_day_features['time_day_features_diversity'] = time_day_features['time_day_features_diff'] / time_day_features['time_day_features_count']
    
    diff_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(lambda x: distance_calc(x.tolist()))
    time_day_features['max_diff'] = diff_features.apply(lambda x: max(x) if x != []  else -100)
    time_day_features['min_diff'] = diff_features.apply(lambda x: min(x) if x != []  else -100)
    time_day_features['mean_diff'] = diff_features.apply(lambda x: np.mean(x) if x != []  else -100)
    time_day_features['std_diff'] = diff_features.apply(lambda x: np.std(x) if x != []  else -100)
    time_day_features['hard_time_step_polyfit'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[1] if len(x) > 2  else -100)
    time_day_features['hard_time_step_polyfit_st'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 2  else -100)

    return time_day_features

time_features = get_time_features(df_train_cutoff)

def get_src_code_features(df):
        src_embeds = pd.pivot_table(
                             df,
                             index =['clientbankpartner_pin'],
                             values=['days_to_report'],
                             columns=['partner_src_type_ccode'],
                             aggfunc=['count','mean','max','std','min']).fillna(0).sort_index()
        
        src_embeds.columns = [f'src_embeds_{i}' for i in src_embeds.columns]
        
        return src_embeds

src_features = get_src_code_features(df_train_cutoff)

def partnerrolestart_date_feature(df,report_date = '2020-9-01'):
    df['days_from_start_to_report'] = pd.to_datetime(report_date) - df['partnerrolestart_date']
    start_day_features = df.groupby('clientbankpartner_pin')['days_from_start_to_report'].agg(['min'])
    start_day_features['days_from_start_to_report'] = start_day_features['min'].dt.days
    return start_day_features.drop(['min'],axis=1)
start_day_features = partnerrolestart_date_feature(df_train_cutoff)

def get_user_features(df):
    user_strs = df.groupby('clientbankpartner_pin')['client_pin'].agg(lambda x:x.tolist())
    user_features = pd.DataFrame()
    user_features['user_nunique'] = user_strs.apply(lambda x: len(np.unique(x)))
    user_features['user_diversity'] = user_features['user_nunique'] / user_strs.apply(len)
    
    user_data = df.groupby('client_pin')['clientbankpartner_pin'].agg(['count'])
    user_data['client_pin'] = user_data.index
    user_data = user_data.rename({'count':'count_tr_per_user'},axis=1)
    user_data.index = range(len(user_data))
    df = df.merge(user_data)
    
    user_features['user_max_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('max')
    return user_features

user_features = get_user_features(df_train_cutoff)

def last_count(df,bins=[5,10,15,20,25,30,35,45,50,60]):
    feature_df = pd.DataFrame(index=df.groupby('clientbankpartner_pin')['client_pin'].agg('count').index)
    for x in bins:
        feature_df[f'count_{x}_prev'] = df[df['days_to_report'] > x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'count_{x}_last'] = df[df['days_to_report'] <= x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'percent_{x}'] =  feature_df[f'count_{x}_last'] / (feature_df[f'count_{x}_last'] + feature_df[f'count_{x}_prev'])
        
    return feature_df.fillna(-1)

last_count_feats = last_count(df_train_cutoff)


all_data = pd.concat([time_features,
                      src_features,
                      start_day_features,
                      last_count_feats,
                      user_features,
                      ],axis=1)

/tmp/ipykernel_34/1039882320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['client_start_date'].dt.month
/tmp/ipykernel_34/1039882320.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'] = df['client_start_date'].dt.day
/tmp/ipykernel_34/1039882320.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [40]:
data_train = data_train.sort_values(by='partner_pin')

In [41]:
all_data = all_data.sort_index()

In [42]:
data_train = pd.concat([all_data.reset_index(),data_train.reset_index()],ignore_index=False,axis=1)

In [44]:
positive_scores = df[df['client_start_date'] >= cutoff_train_date]['clientbankpartner_pin'].unique()
train_score = data_train['partner_pin'].apply(lambda x: 0 if x in positive_scores else 1)
drop_on_fitting = {'columns': ["partner_pin", "lst_dates_involving", "partner_start_date"]}

In [45]:
train_pool = Pool(data=data_train.drop(**drop_on_fitting),
                  label=train_score)

In [46]:
model_params = {
    "loss_function":'CrossEntropy',
    "iterations": 1500,
    "learning_rate": 0.01,
    "max_depth": 4,
    #"l2_leaf_reg": 5,
    #"bootstrap_type": "Bernoulli",
    #"subsample": 0.8,
    "random_seed": 5656,
    "eval_metric": "AUC",
    #"early_stopping_rounds": 100,
}

cv_data,models = cv(train_pool,
                    params=model_params,
                    nfold=5,
                    seed=5656,
                    stratified=True,
                    shuffle=True,
                    return_models=True,
                    plot=True,
                    verbose=200)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	test: 0.8171250	best: 0.8171250 (0)	total: 11.4ms	remaining: 17.1s
200:	test: 0.8434813	best: 0.8434813 (200)	total: 2.03s	remaining: 13.1s
400:	test: 0.8463182	best: 0.8463312 (398)	total: 3.86s	remaining: 10.6s
600:	test: 0.8475121	best: 0.8475362 (599)	total: 5.64s	remaining: 8.43s
800:	test: 0.8477737	best: 0.8478736 (776)	total: 7.5s	remaining: 6.55s
1000:	test: 0.8476971	best: 0.8479631 (875)	total: 9.38s	remaining: 4.67s
1200:	test: 0.8474604	best: 0.8479631 (875)	total: 11.3s	remaining: 2.81s
1400:	test: 0.8475319	best: 0.8479631 (875)	total: 13.4s	remaining: 944ms
1499:	test: 0.8474716	best: 0.8479631 (875)	total: 14.2s	remaining: 0us

bestTest = 0.8479630921
bestIteration = 875

Training on fold [1/5]
0:	test: 0.8574655	best: 0.8574655 (0)	total: 14.4ms	remaining: 21.6s
200:	test: 0.8760193	best: 0.8760563 (199)	total: 2.13s	remaining: 13.8s
400:	test: 0.8773250	best: 0.8773750 (397)	total: 4.16s	remaining: 11.4s
600:	test: 0.8776056	best: 0.8777296 

In [47]:
cv_data['test-AUC-mean'].max()

0.8657432952465008

In [48]:
models[0].get_feature_importance(prettified=True)

Feature Id  Importances
0                            since_last_client    11.973665
1                        time_day_features_min     9.989044
2                  time_day_features_diversity     4.584665
3                                count_60_last     3.824678
4                       time_day_features_mean     3.418660
..                                         ...          ...
157                                    week_13     0.000016
158  src_embeds_('count', 'days_to_report', 3)     0.000000
159   src_embeds_('mean', 'days_to_report', 0)     0.000000
160    src_embeds_('std', 'days_to_report', 0)     0.000000
161    src_embeds_('min', 'days_to_report', 3)     0.000000

[162 rows x 2 columns]

In [49]:
def weeks_count_full(lst_dates: list) -> pd.Series:
    lst_weeks = [0 for _ in range(num_weeks + (cutoff_test_date - cutoff_train_date).days // 7)]
    for item in lst_dates:
        lst_weeks[(item - min_date).days // 7] += 1
    return pd.Series(lst_weeks[13:])

In [50]:
data_test = (
    df.groupby("clientbankpartner_pin")
    .agg(
        {
            "client_pin": "count",
            "partner_src_type_ccode": "first",
            "partnerrolestart_date": "first",
            "client_start_date": lambda x: list(x),
        }
    )
    .reset_index()
)

data_test.rename(
    {
        "clientbankpartner_pin": "partner_pin",
        "client_pin": "num_clients",
        "partner_src_type_ccode": "partner_type",
        "partnerrolestart_date": "partner_start_date",
        "client_start_date": "lst_dates_involving",
    },
    axis=1,
    inplace=True,
)

time_series_test = data_test["lst_dates_involving"].apply(weeks_count_full)
time_series_test.columns = [f"week_{i+1}" for i in range(79)]
data_test = pd.concat([data_test, time_series_test], axis=1)

data_test = data_test.assign(
    time_as_partner=(cutoff_test_date - data_test["partner_start_date"]).dt.days,
    since_last_client=data_test["lst_dates_involving"].apply(
        lambda x: (cutoff_test_date - max(x)).days
    ),
    max_length=data_test.apply(max_length, axis=1),
    max_length_increasing=data_test.apply(max_length_increasing, axis=1),
    max_length_zeros=data_test.apply(max_length_zeros, axis=1),
    recent_clients=data_test.apply(recent_clients, axis=1),
)


In [51]:
def create_time_features(df,report_date = '2020-12-01'):
    df['month'] = df['client_start_date'].dt.month
    df['day'] = df['client_start_date'].dt.day
    df['week'] = df['client_start_date'].dt.day.map(lambda x: x // 7)
    df['minute'] = df['client_start_date'].dt.minute
    df['year'] = df['client_start_date'].dt.year

    df['dayofweek'] = df['client_start_date'].dt.dayofweek
    df['all_year_time'] = df['client_start_date'].apply(lambda x: (x.year - 2019) * 365 + x.month * 30 + x.day)
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')
    
    df['time_to_report'] = pd.to_datetime(report_date) - df['client_start_date']
    df['days_to_report'] = df['time_to_report'].dt.days.astype(int)
    
    return df

df = create_time_features(df)

def get_time_features(df):
    time_day_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(['mean', 'std', 'min', 'max', 'median','count'])
    time_day_features.columns = [f'time_day_features_{c}' for c in time_day_features.columns]
    time_day_features['time_day_features_diff'] = time_day_features['time_day_features_max'] - time_day_features['time_day_features_min']
    time_day_features['time_day_features_diversity'] = time_day_features['time_day_features_diff'] / time_day_features['time_day_features_count']
    
    diff_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(lambda x: distance_calc(x.tolist()))
    time_day_features['max_diff'] = diff_features.apply(lambda x: max(x) if x != []  else -100)
    time_day_features['min_diff'] = diff_features.apply(lambda x: min(x) if x != []  else -100)
    time_day_features['mean_diff'] = diff_features.apply(lambda x: np.mean(x) if x != []  else -100)
    time_day_features['std_diff'] = diff_features.apply(lambda x: np.std(x) if x != []  else -100)
    time_day_features['hard_time_step_polyfit'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[1] if len(x) > 2  else -100)
    time_day_features['hard_time_step_polyfit_st'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 2  else -100)

    return time_day_features

time_features = get_time_features(df)

def get_src_code_features(df):
        src_embeds = pd.pivot_table(
                             df,
                             index =['clientbankpartner_pin'],
                             values=['days_to_report'],
                             columns=['partner_src_type_ccode'],
                             aggfunc=['count','mean','max','std','min']).fillna(0).sort_index()
        
        src_embeds.columns = [f'src_embeds_{i}' for i in src_embeds.columns]
        
        return src_embeds

src_features = get_src_code_features(df)

def partnerrolestart_date_feature(df,report_date = '2020-12-01'):
    df['days_from_start_to_report'] = pd.to_datetime(report_date) - df['partnerrolestart_date']
    start_day_features = df.groupby('clientbankpartner_pin')['days_from_start_to_report'].agg(['min'])
    start_day_features['days_from_start_to_report'] = start_day_features['min'].dt.days
    return start_day_features.drop(['min'],axis=1)
start_day_features = partnerrolestart_date_feature(df)

def get_user_features(df):
    user_strs = df.groupby('clientbankpartner_pin')['client_pin'].agg(lambda x:x.tolist())
    user_features = pd.DataFrame()
    user_features['user_nunique'] = user_strs.apply(lambda x: len(np.unique(x)))
    user_features['user_diversity'] = user_features['user_nunique'] / user_strs.apply(len)
    
    user_data = df.groupby('client_pin')['clientbankpartner_pin'].agg(['count'])
    user_data['client_pin'] = user_data.index
    user_data = user_data.rename({'count':'count_tr_per_user'},axis=1)
    user_data.index = range(len(user_data))
    df = df.merge(user_data)
    
    user_features['user_max_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('max')
    return user_features

user_features = get_user_features(df)

def last_count(df,bins=[5,10,15,20,25,30,35,45,50,60]):
    feature_df = pd.DataFrame(index=df.groupby('clientbankpartner_pin')['client_pin'].agg('count').index)
    for x in bins:
        feature_df[f'count_{x}_prev'] = df[df['days_to_report'] > x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'count_{x}_last'] = df[df['days_to_report'] <= x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'percent_{x}'] =  feature_df[f'count_{x}_last'] / (feature_df[f'count_{x}_last'] + feature_df[f'count_{x}_prev'])
        
    return feature_df.fillna(-1)

last_count_feats = last_count(df)

all_data_test = pd.concat([time_features,
                      src_features,
                      start_day_features,
                      last_count_feats,
                      user_features,
                      ],axis=1)

In [52]:
data_test = data_test.sort_values(by='partner_pin')
all_data_test = all_data_test.sort_index()
data_test = pd.concat([all_data_test.reset_index(),data_test.reset_index()],ignore_index=False,axis=1)

In [53]:
data_test["score"] = np.mean([cbm.predict(data_test.drop(**drop_on_fitting).astype(float)) for cbm in models],axis=0)

In [54]:
from math import e
def sigmoid(x):
    return 1 / (1 + e ** (-x))

In [55]:
data_test["score"] = data_test['score'].map(sigmoid)

In [57]:
sub = pd.read_csv('submission_example.csv')
sub['score'] = sub['clientbankpartner_pin'].apply(lambda x: data_test[data_test['partner_pin'] == x]['score'].iloc[0])
sub.to_csv('CatBoostBaselineV2.csv', index=False)
sub

clientbankpartner_pin     score
0                      6781  0.029510
1                    236905  0.063990
2                    125779  0.205932
3                      1952  0.185221
4                      4872  0.425505
...                     ...       ...
4503                 121120  0.347596
4504                  60667  0.658772
4505                   5065  0.695313
4506                 133125  0.327369
4507                 121963  0.498165

[4508 rows x 2 columns]